In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 30
import os
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import datetime
import pymysql
import time
from datetime import date
import openpyxl
import glob
import shutil

# Selenium libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select



print("Importado con éxito")

Importado con éxito


In [22]:
try:
    connection = pymysql.connect(host="192.168.100.215",user="root", passwd="sqladmin", db="siogranos", port=3316)
    cursor = connection.cursor()
    base_de_datos = pd.read_sql_query (f"SELECT * FROM data", connection)
    connection.commit()
    connection.close()
    print("La base se ha cargado correctamente")
except:
    print("No se encontró la base de datos del SIO-GRANOS")

C:\Users\frolotti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


La base se ha cargado correctamente


In [30]:
# Borramos los archivos del directorio en el que descargaremos los datos que faltan

directory = 'Archivos/*'
files = glob.glob(directory)

for file in files:
    if os.path.exists(file):
        os.remove(file)

In [41]:
base_de_datos

,FECHA OPERACION,FECHA CONCERTACION,OPERACION,TIPO,PRECIO,PRODUCTO,CANT. (TN),CALIDAD,CALIDAD ADICIONAL,PROCEDENCIA PCIA,PROCEDENCIA LOCALID.,PRECIO/TN MONEDA,PRECIO/TN MONTO,LUGAR ENTREGA,FECHA ENTR. DESDE,FECHA ENTR. HASTA,CONDICION PAGO,ES FINAL,COSECHA,NRO INSTANCIA OPERACION,Unnamed: 20
578529,26/08/2021 04:15:39 p.m.,2002-01-08,Contrato,Compraventa,Precio Hecho,MAIZ,60.37,Otra,nan,CÓRDOBA,CALASUYA,$,17860.0,Zona 11/En destino,01/08/2021 12:00:00 a.m.,30/08/2021 12:00:00 a.m.,Contra entrega,SI,COSECHA 20/21,1.0,nan
1288449,29/12/2022 09:32:57 a.m.,2004-12-28,Contrato,Compraventa,Precio Hecho,SOJA,200.0,Fábrica,nan,CÓRDOBA,ADELIA MARIA,U$S,149.48,Rosario N/En destino,01/04/2005 12:00:00 a.m.,31/05/2005 12:00:00 a.m.,Contra entrega,SI,COSECHA 04/05,1.0,nan
1288447,29/12/2022 09:33:02 a.m.,2004-12-28,Contrato,Compraventa,Fijar Precio/Mercado Comprador,SOJA,300.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005 12:00:00 a.m.,30/05/2005 12:00:00 a.m.,A plazo,SI,COSECHA 04/05,1.0,nan
1288422,29/12/2022 09:41:09 a.m.,2005-01-04,Contrato,Compraventa,Precio Hecho,SOJA,60.0,Fábrica,nan,CÓRDOBA,VIAMONTE,$,446.31,Zona 8/En destino,01/04/2005 12:00:00 a.m.,02/05/2005 12:00:00 a.m.,Contra entrega,SI,COSECHA 04/05,1.0,nan
1288424,29/12/2022 09:41:05 a.m.,2005-01-04,Contrato,Compraventa,Precio Hecho,SOJA,70.0,Fábrica,nan,CÓRDOBA,JESUS MARIA,$,475.1,Rosario N/En destino,01/04/2005 12:00:00 a.m.,01/05/2005 12:00:00 a.m.,Contra entrega,SI,COSECHA 04/05,1.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1865317,01/02/2023 11:00:02 a.m.,2023-12-01,Contrato,Canje,Fijar Precio/Otros,SOJA,0.0,Cámara,nan,CÓRDOBA,SERRANO,0,13171.68,Rosario S/En destino,12/01/2023 12:00:00 a.m.,15/05/2023 12:00:00 a.m.,Anticipado a la entrega,NO,COSECHA 22/23,1,None
1265384,12/01/2023 05:47:01 p.m.,2023-12-01,Contrato,Canje,Precio Hecho,SOJA,25.96,Fábrica,nan,CÓRDOBA,COLONIA LAS CUATRO E,$,99000.0,Zona 11/En origen,12/01/2023 12:00:00 a.m.,12/01/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1.0,nan
1265387,12/01/2023 05:46:13 p.m.,2023-12-01,Contrato,Compraventa,Precio Hecho,SOJA,30.0,Fábrica,nan,CÓRDOBA,CAMILO ALDAO,$,91000.0,Rosario N/En destino,12/01/2023 12:00:00 a.m.,12/02/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1.0,nan
1265395,12/01/2023 05:42:07 p.m.,2023-12-01,Contrato,Compraventa,Precio Hecho,SOJA,10.0,Cámara,nan,CÓRDOBA,INRIVILLE,$,75000.0,Zona 8/En destino,12/01/2023 12:00:00 a.m.,11/02/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 21/22,1.0,nan


In [55]:

base_de_datos['FECHA CONCERTACION'] = pd.to_datetime(base_de_datos['FECHA CONCERTACION'])
base_de_datos['FECHA CONCERTACION'] = base_de_datos['FECHA CONCERTACION'].dt.strftime('%Y-%m-%d')
base_de_datos = base_de_datos.sort_values(by='FECHA CONCERTACION')
base_de_datos

C:\Users\frolotti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2005' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\frolotti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\frolotti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2015' in DD/MM/YYYY format. Provide

,FECHA OPERACION,FECHA CONCERTACION,OPERACION,TIPO,PRECIO,PRODUCTO,CANT. (TN),CALIDAD,CALIDAD ADICIONAL,PROCEDENCIA PCIA,PROCEDENCIA LOCALID.,PRECIO/TN MONEDA,PRECIO/TN MONTO,LUGAR ENTREGA,FECHA ENTR. DESDE,FECHA ENTR. HASTA,CONDICION PAGO,ES FINAL,COSECHA,NRO INSTANCIA OPERACION,Unnamed: 20
578529,26/08/2021 04:15:39 p.m.,2002-08-01,Contrato,Compraventa,Precio Hecho,MAIZ,60.37,Otra,nan,CÓRDOBA,CALASUYA,$,17860.0,Zona 11/En destino,01/08/2021 12:00:00 a.m.,30/08/2021 12:00:00 a.m.,Contra entrega,SI,COSECHA 20/21,1.0,nan
1288449,29/12/2022 09:32:57 a.m.,2004-12-28,Contrato,Compraventa,Precio Hecho,SOJA,200.0,Fábrica,nan,CÓRDOBA,ADELIA MARIA,U$S,149.48,Rosario N/En destino,01/04/2005 12:00:00 a.m.,31/05/2005 12:00:00 a.m.,Contra entrega,SI,COSECHA 04/05,1.0,nan
1288447,29/12/2022 09:33:02 a.m.,2004-12-28,Contrato,Compraventa,Fijar Precio/Mercado Comprador,SOJA,300.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005 12:00:00 a.m.,30/05/2005 12:00:00 a.m.,A plazo,SI,COSECHA 04/05,1.0,nan
1288443,29/12/2022 09:33:12 a.m.,2005-01-13,Contrato,Compraventa,Precio Hecho,SOJA,30.0,Fábrica,nan,CÓRDOBA,JESUS MARIA,U$S,143.0,Rosario N/En destino,01/03/2005 12:00:00 a.m.,30/05/2005 12:00:00 a.m.,Contra entrega,SI,COSECHA 04/05,1.0,nan
1288448,29/12/2022 09:33:01 a.m.,2005-01-23,Contrato,Compraventa,Fijar Precio/Mercado Comprador,SOJA,700.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005 12:00:00 a.m.,30/05/2005 12:00:00 a.m.,A plazo,SI,COSECHA 04/05,1.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853412,15/02/2023 06:25:16 p.m.,2023-02-15,Contrato,Canje,Precio Hecho,MAIZ,60.0,Cámara,nan,CÓRDOBA,COLONIA ARGENTINA,U$S,259.38,Rosario S/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1,None
1854234,15/02/2023 10:11:03 a.m.,2023-02-15,Contrato,Compraventa,Precio Hecho,MAIZ,385.64,Fábrica,nan,CÓRDOBA,COLONIA VEINTICINCO,$,44353.13,Zona 1/En destino,15/02/2023 12:00:00 a.m.,15/02/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 21/22,1,None
1853427,15/02/2023 06:15:31 p.m.,2023-02-15,Contrato,Compraventa,Precio Hecho,MAIZ,300.0,Cámara,nan,CÓRDOBA,CAMPO DE LAS PIEDRAS,$,51000.0,Zona 8/En destino,15/02/2023 12:00:00 a.m.,15/02/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1,None
1854074,15/02/2023 11:19:54 a.m.,2023-02-15,Contrato,Compraventa,Precio Hecho,MAIZ,1.65,Cámara,nan,CÓRDOBA,JUSTINIANO POSSE,$,49800.0,Zona 8/En origen,15/02/2023 12:00:00 a.m.,15/02/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1,None


In [56]:
old_max_day = 1
old_max_month = 6
old_max_year = 2022
if date.today().day==1:                  # El día corriente no está disponible para descargar, hay que
    today_day = 28                       # usar el día anterior
    if date.today().month==1:
        today_month = 11
        today_year = date.today().year-1
    else:
        today_month = date.today().month-2
else:    
    today_day = date.today().day-1
    today_month = date.today().month-1       # Enero lo cuentan como 0, y de ahí empiezan a sumar...
    today_year = date.today().year


In [58]:
base_de_datos['FECHA CONCERTACION'].iloc[-1]

'2023-02-15'

In [66]:
# Fechas para la descarga de datos

old_max_day = int(base_de_datos['FECHA CONCERTACION'].iloc[-1][8:10])+1
old_max_month = int(base_de_datos['FECHA CONCERTACION'].iloc[-1][5:7])-1
old_max_year = int(base_de_datos['FECHA CONCERTACION'].iloc[-1][0:4])




if date.today().day==1:                  # El día corriente no está disponible para descargar, hay que
    today_day = 28                       # usar el día anterior
    if date.today().month==1:
        today_month = 11
        today_year = date.today().year-1
    else:
        today_month = date.today().month-2
else:    
    today_day = date.today().day-1
    today_month = date.today().month-1      # Enero lo cuentan como 0, y de ahí empiezan a sumar...
    today_year = date.today().year



procedencia = 'CÓRDOBA'


print("Fechas de inicio y fin a descargar: ")
print(old_max_day, old_max_month+1, old_max_year)
print(today_day,today_month+1,today_year)

Fechas de inicio y fin a descargar: 
16 2 2023
21 2 2023


In [64]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.siogranos.com.ar/Consulta_publica/operaciones_informadas_exportar.aspx")
driver.maximize_window()
time.sleep(3)




# Fecha de Inicio:
fecha_inicio = driver.find_element(By.NAME,"txtFechaOperacionDesde").click()
time.sleep(1)
# Mes:
select_mes_inicio = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-month"))
select_mes_inicio.select_by_value(str(old_max_month))
time.sleep(1)
# Año:
select_anio_inicio = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-year"))
select_anio_inicio.select_by_value(str(old_max_year))
time.sleep(1)
# Dia:
day_inicio = driver.find_element(By.XPATH,"//a[@class='ui-state-default' and text()="+str(old_max_day)+"]")
day_inicio.click()


# Fecha de hoy:
fecha_hoy = driver.find_element(By.NAME,"txtFechaOperacionHasta").click()
time.sleep(1)
# Mes:
select_mes = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-month"))
select_mes.select_by_value(str(today_month))
time.sleep(1)
# Año:
select_anio = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-year"))
select_anio.select_by_value(str(today_year))
time.sleep(1)
# Dia:
day = driver.find_element(By.XPATH,"//a[@class='ui-state-default' and text()="+str(today_day)+"]")
day.click()

# Procedencia:
sel = Select(driver.find_element(By.ID, "ddlProvincia"))
sel.select_by_visible_text(procedencia)



csv = driver.find_element(By.ID,"btn_generar_csv")
csv.click()

#fecha_hasta = driver.find_element(By.NAME,"txtFechaOperacionHasta").click()




In [67]:
folder_path = r'C:\Users\frolotti\Downloads'
file_type = r'\*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

print(max_file)

C:\Users\frolotti\Downloads\operaciones_informadas (2).csv


In [68]:
shutil.move(max_file, r'C:\Users\frolotti\Documents\PythonProjects\siogranos\SIO-GRANOS\sio-granos\Archivos')

'C:\\Users\\frolotti\\Documents\\PythonProjects\\siogranos\\SIO-GRANOS\\sio-granos\\Archivos\\operaciones_informadas (2).csv'

In [73]:
# Este código para agregar nuevos datos a la base SQL, no usarlo si no hay datos a agregar.
# Ingresar a: https://www.siogranos.com.ar/Consulta_publica/operaciones_informadas_exportar.aspx
# Descargar los días a agregar con filtro de Cordoba como procedencia Provincia y guardarlos en la carpeta de SIO-GRANOS.

# Creo un Dataframe vacio y una lista con los archivos que se encuentran en el directorio, en caso de querer 
# agregar un período de tiempo muy largo:
data = pd.DataFrame()
fichero = os.listdir('Archivos/')

#Creo un "for" que me lea todos los archivos y me vaya apendizando las bases a nuestro Dataframe "data":

for base in fichero:
    if (base!='.ipynb_checkpoints' and base!='SIO-GRANOS.ipynb' and base!='Dolar.xlsx' and base!='Archivos'):    # Compila todos los archivos de la carpeta excepto estos dos.
        apendice = pd.read_excel(f"Archivos/{base}")
        data = pd.concat([data,apendice])


data

,FECHA OPERACION,FECHA CONCERTACION,OPERACION,TIPO,PRECIO,PRODUCTO,CANT. (TN),CALIDAD,CALIDAD ADICIONAL,PROCEDENCIA PCIA,PROCEDENCIA LOCALID.,PRECIO/TN MONEDA,PRECIO/TN MONTO,LUGAR ENTREGA,FECHA ENTR. DESDE,FECHA ENTR. HASTA,CONDICION PAGO,ES FINAL,COSECHA,NRO INSTANCIA OPERACION
0,21/02/2023 02:04:21 p.m.,21/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,TRIGO PAN,30.01,Cámara,NaN,CÓRDOBA,LAS VARILLAS,$,60000.0,Cordoba/En destino,21/02/2023 12:00:00 a.m.,21/02/2023 12:00:00 a.m.,A plazo,SI,COSECHA 22/23,1
1,20/02/2023 10:24:41 a.m.,17/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,MAIZ,30.00,Cámara,NaN,CÓRDOBA,VILLA DEL ROSARIO,$,51142.0,Zona 14/En destino,17/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 21/22,1
2,18/02/2023 11:39:24 a.m.,18/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,MAIZ,4.07,Cámara,NaN,CÓRDOBA,JUSTINIANO POSSE,$,50100.0,Zona 8/En origen,18/02/2023 12:00:00 a.m.,18/02/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
3,18/02/2023 11:39:21 a.m.,18/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,MAIZ,76.50,Cámara,NaN,CÓRDOBA,JUSTINIANO POSSE,$,50100.0,Zona 8/En origen,18/02/2023 12:00:00 a.m.,18/02/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
4,18/02/2023 11:39:19 a.m.,18/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,MAIZ,30.00,Cámara,NaN,CÓRDOBA,JUSTINIANO POSSE,$,50100.0,Zona 8/En origen,18/02/2023 12:00:00 a.m.,18/02/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,16/02/2023 08:04:51 a.m.,15/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,SOJA,65.00,Fábrica,NaN,CÓRDOBA,PASCANAS,$,90500.0,Rosario N/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
1992,16/02/2023 08:04:35 a.m.,15/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,SOJA,30.00,Fábrica,NaN,CÓRDOBA,ETRURIA,$,90500.0,Rosario N/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
1993,16/02/2023 08:04:19 a.m.,15/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,MAIZ,105.00,Cámara,NaN,CÓRDOBA,ETRURIA,U$S,260.0,Rosario N/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
1994,16/02/2023 08:02:46 a.m.,15/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,MAIZ,180.00,Cámara,NaN,CÓRDOBA,COLONIA ALMADA,U$S,260.0,Rosario N/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1


In [74]:
baseSQL = pd.DataFrame()

for i in data.columns:
    baseSQL[i] = data[i].astype(str)


# Busco los nombres de columnas y paso el tipo de dato del dataframe a uno compatible con SQL:    
    
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar(50)')
    return dataList

columnas_nombres = list(baseSQL.columns.values)
columnas_tipo_de_datos = getColumnDtypes(baseSQL.dtypes)




In [75]:


# Defino el statement para crear la tabla:
crear_tabla_statement = 'CREATE TABLE IF NOT EXISTS data ('
for i in range(len(columnas_tipo_de_datos)):
    crear_tabla_statement = crear_tabla_statement +"\n`" + columnas_nombres[i] +  "` " + columnas_tipo_de_datos[i] + ', '
crear_tabla_statement = crear_tabla_statement[:-2] + ');'
    
    # Me conecto a la base

connection = pymysql.connect(host="192.168.100.215",user="root", passwd="sqladmin", db="siogranos", port=3316)
cursor = connection.cursor()

    # Creo la tabla:
try: 
    cursor.execute(crear_tabla_statement)
except:
    print("La tabla ya existe, se va a proceder a cargar los datos:")


    # Preparo la inserción de datos:
cols = "`,`".join([str(i) for i in baseSQL.columns.tolist()])

for i,row in baseSQL.iterrows():
    sql = "INSERT INTO data (`" + cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))


#cursor.execute("UPDATE data SET `Fecha` = STR_TO_DATE(`FECHA OPERACION`,'%Y-%m-%d') where Fecha IS NULL;")

connection.commit()
connection.close()
print("Carga exitosa.")

Carga exitosa.
